# മോഡൽ കോൺടെക്സ്റ്റ് പ്രോട്ടോക്കോൾ (MCP) പിന്തുണയുള്ള Azure AI ഏജന്റുകൾ

ബാഹ്യ MCP സർവറുകൾ ഉപയോഗിച്ച് മെച്ചപ്പെട്ട കഴിവുകൾ നേടാൻ കഴിയുന്ന ബുദ്ധിമുട്ടുള്ള ഏജന്റ് സൃഷ്ടിക്കാൻ മോഡൽ കോൺടെക്സ്റ്റ് പ്രോട്ടോക്കോൾ (MCP) ടൂളുകളുമായി Azure AI ഏജന്റുകൾ എങ്ങനെ ഉപയോഗിക്കാമെന്ന് ഈ നോട്ട്‌ബുക്ക് കാണിക്കുന്നു.


## Install Required NuGet Packages

ആദ്യം, Azure AI ഏജന്റുകളുമായി പ്രവർത്തിക്കാൻ അടിസ്ഥാന പ്രവർത്തനക്ഷമത നൽകുന്ന Azure AI Agents Persistent പാക്കേജ് ഇൻസ്റ്റാൾ ചെയ്യേണ്ടതാണ്.


## കീലെസ് ഓതന്റിക്കേഷൻ ഗുണങ്ങൾ

ഈ നോട്ട്‌ബുക്ക് **കീലെസ് ഓതന്റിക്കേഷൻ** പ്രദർശിപ്പിക്കുന്നു, ഇത് നിരവധി ഗുണങ്ങൾ നൽകുന്നു:
- ✅ **മാനേജ് ചെയ്യാനുള്ള API കീകൾ ഇല്ല** - Azure ഐഡന്റിറ്റി അടിസ്ഥാനമാക്കിയ ഓതന്റിക്കേഷൻ ഉപയോഗിക്കുന്നു
- ✅ **വളർന്ന സുരക്ഷ** - കോഡ് അല്ലെങ്കിൽ കോൺഫിഗറേഷനിൽ രഹസ്യങ്ങൾ സൂക്ഷിക്കാറില്ല
- ✅ **സ്വയം ക്രെഡൻഷ്യൽ റൊട്ടേഷൻ** - ക്രെഡൻഷ്യൽ ലൈഫ്‌സൈക്കിൾ Azure കൈകാര്യം ചെയ്യുന്നു
- ✅ **റോൾ അടിസ്ഥാനമാക്കിയ ആക്‌സസ്** - സൂക്ഷ്മ അനുമതികൾക്കായി Azure RBAC ഉപയോഗിക്കുന്നു

`DefaultAzureCredential` സ്വയം ലഭ്യമായ മികച്ച ക്രെഡൻഷ്യൽ ഉറവിടം ഉപയോഗിക്കും:
1. മാനേജ്ഡ് ഐഡന്റിറ്റി (Azure-യിൽ പ്രവർത്തിക്കുമ്പോൾ)
2. Azure CLI ക്രെഡൻഷ്യലുകൾ (വികസന സമയത്ത്)
3. Visual Studio ക്രെഡൻഷ്യലുകൾ
4. പരിസ്ഥിതി വേരിയബിളുകൾ (കോണ്ഫിഗർ ചെയ്തിട്ടുണ്ടെങ്കിൽ)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure സേവനങ്ങളുമായി DefaultAzureCredential ഉപയോഗിച്ച് പ്രാമാണീകരണത്തിന് Azure Identity പാക്കേജ് ഇൻസ്റ്റാൾ ചെയ്യുക.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## ആവശ്യമായ നെയിംസ്പേസുകൾ ഇറക്കുമതി ചെയ്യുക

Azure AI ഏജന്റുകൾക്കും Azure ഐഡന്റിറ്റിക്കും ആവശ്യമായ നെയിംസ്പേസുകൾ ഇറക്കുമതി ചെയ്യുക.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI ഏജന്റ് ക്ലയന്റ് കോൺഫിഗർ ചെയ്യുക (കീലെസ് ഓതന്റിക്കേഷൻ)

കോൺഫിഗറേഷൻ വേരിയബിളുകൾ സജ്ജമാക്കി **കീലെസ് ഓതന്റിക്കേഷൻ** ഉപയോഗിച്ച് PersistentAgentsClient സൃഷ്ടിക്കുക:
- **projectEndpoint**: Azure AI Foundry പ്രോജക്ട് എൻഡ്‌പോയിന്റ്
- **modelDeploymentName**: ഡിപ്ലോയുചെയ്ത AI മോഡലിന്റെ പേര് (GPT-4.1 നാനോ)
- **mcpServerUrl**: MCP സെർവറിന്റെ URL (Microsoft Learn API)
- **mcpServerLabel**: MCP സെർവർ തിരിച്ചറിയാനുള്ള ലേബൽ
- **DefaultAzureCredential**: മാനേജ്ഡ് ഐഡന്റിറ്റി, Azure CLI, അല്ലെങ്കിൽ മറ്റ് ക്രെഡൻഷ്യൽ സ്രോതസ്സുകൾ ഉപയോഗിക്കുന്നു (API കീകൾ ആവശ്യമില്ല)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP ടൂൾ നിർവചനം സൃഷ്ടിക്കുക

Microsoft Learn MCP സെർവറുമായി ബന്ധിപ്പിക്കുന്ന MCP ടൂൾ നിർവചനം സൃഷ്ടിക്കുക. ഇത് ഏജന്റിന് Microsoft Learn ഉള്ളടക്കം மற்றும் ഡോക്യുമെന്റേഷൻ ആക്‌സസ് ചെയ്യാൻ അനുവദിക്കും.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI ഏജന്റ് സൃഷ്ടിക്കുക

നിർദ്ദിഷ്ട മോഡലും MCP ഉപകരണങ്ങളും ഉപയോഗിച്ച് സ്ഥിരതയുള്ള AI ഏജന്റ് സൃഷ്ടിക്കുക. ഏജന്റ് താഴെപ്പറയുന്നവയുമായി ക്രമീകരിച്ചിരിക്കുന്നു:
- GPT-4.1 നാനോ മോഡൽ
- സഹായത്തിനായി MCP ഉപകരണങ്ങൾ ഉപയോഗിക്കാനുള്ള നിർദ്ദേശങ്ങൾ
- Microsoft Learn MCP സെർവറിലേക്ക് ആക്‌സസ്


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## ത്രെഡ് സൃഷ്ടിച്ച് സന്ദേശം അയയ്ക്കുക

ഒരു സംഭാഷണ ത്രെഡ് സൃഷ്ടിച്ച് Azure OpenAIയും OpenAIയും തമ്മിലുള്ള വ്യത്യാസത്തെക്കുറിച്ച് ചോദിക്കുന്ന ഒരു ഉപയോക്തൃ സന്ദേശം അയയ്ക്കുക. ഇത് ഏജന്റിന്റെ MCP ഉപകരണങ്ങൾ ഉപയോഗിച്ച് കൃത്യമായ വിവരങ്ങൾ നൽകാനുള്ള കഴിവ് പരിശോധിക്കും.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP ടൂൾ റിസോഴ്‌സുകൾ കോൺഫിഗർ ചെയ്യുക (കീലെസ്)

MCP ടൂൾ റിസോഴ്‌സുകൾ സജ്ജമാക്കുക. യഥാർത്ഥ കീലെസ് സമീപനത്തിനായി, MCP സെർവർ Azure ഐഡന്റിറ്റി അടിസ്ഥാനമാക്കിയുള്ള ഓതന്റിക്കേഷൻ പിന്തുണയ്ക്കുന്നുവെങ്കിൽ, കസ്റ്റം ഹെഡറുകൾ നീക്കം ചെയ്യാം. താഴെയുള്ള ഉദാഹരണം ആവശ്യമായാൽ ഹെഡറുകൾ എങ്ങനെ ചേർക്കാമെന്ന് കാണിക്കുന്നു, പക്ഷേ കീലെസ് സാഹചര്യങ്ങളിൽ ഇവ ആവശ്യമായിരിക്കില്ല.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## ഏജന്റ് റൺ ആരംഭിക്കുക

ഉപയോക്താവിന്റെ സന്ദേശം പ്രോസസ്സ് ചെയ്യാൻ ഒരു റൺ സൃഷ്ടിച്ച് ആരംഭിക്കുക. ഏജന്റ് പ്രതികരണം സൃഷ്ടിക്കാൻ ക്രമീകരിച്ച MCP ഉപകരണങ്ങളും വിഭവങ്ങളും ഉപയോഗിക്കും.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## റൺ നിരീക്ഷിക്കുക, ടൂൾ അംഗീകാരങ്ങൾ കൈകാര്യം ചെയ്യുക (കീലെസ്)

ഏജന്റ് റൺ നില നിരീക്ഷിക്കുകയും ആവശ്യമായ ടൂൾ അംഗീകാരങ്ങൾ കൈകാര്യം ചെയ്യുകയും ചെയ്യുക. ഈ ലൂപ്പ്:
1. റൺ പൂർത്തിയാകുന്നതോ പ്രവർത്തനം ആവശ്യപ്പെടുന്നതോ വരെ കാത്തിരിക്കുന്നു
2. ആവശ്യമായപ്പോൾ MCP ടൂൾ കോൾസ് സ്വയം അംഗീകരിക്കുന്നു
3. കീലെസ് ഓതന്റിക്കേഷനിനായി, MCP സെർവർ Azure ഐഡന്റിറ്റി പിന്തുണയ്ക്കുന്നുവെങ്കിൽ ഹെഡറുകൾ ആവശ്യമില്ലായിരിക്കും


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## സംഭാഷണ ഫലങ്ങൾ പ്രദർശിപ്പിക്കുക

തന്ത്രത്തിൽ ഉള്ള എല്ലാ സന്ദേശങ്ങളും തിരികെ കൊണ്ടുവന്ന് പ്രദർശിപ്പിക്കുക, ഉപയോക്താവിന്റെ ചോദ്യം കൂടാതെ ഏജന്റിന്റെ പ്രതികരണവും കാണിക്കുന്നു. സന്ദേശങ്ങൾ സമയക്രമത്തിൽ ടൈംസ്റ്റാമ്പുകളും റോളും സൂചിപ്പിക്കുന്ന അടയാളങ്ങളോടുകൂടി പ്രദർശിപ്പിക്കുന്നു.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
